In [42]:
# day 12 - part 1
# define a function that processes the input
def cleanInput(inputFilename):
  with open(inputFilename, 'r') as file:
    inputData = file.read()

  # general cleaning
  inputData = inputData.split('\n')
  inputData = list(filter(None, inputData))

  # problem-specific cleaning
  temp = []
  for line in inputData:
    line = line.split(' ')
    springs, groups = line[0], line[1]
    ###
    # you must use tuple here to use functools because list is unhashable, because list is mutable
    springs = tuple(char for char in springs)
    groups = tuple(int(num) for num in groups.split(','))
    ## (past notes) or we can just use list(springs) (should i use tuple here?)
    ## springs = [char for char in springs]
    ## groups = [int(num) for num in groups.split(',')]
    ###
    temp.append([springs, groups])
  inputData = temp
  ###
  return inputData

inputFilename = 'advent-of-code-day12-input'
# inputFilename = 'advent-of-code-day12-testcase'
springsRows = cleanInput(inputFilename)
# springsRows

In [43]:
# solution
# inspiration from https://cutonbuminband.github.io/AOC/qmd/2023.html#day-12-hot-springs
# almost all the logic is from there
import functools

def checkFirstGroup(springs, group): # check the first chunk/group if it's viable (ie. a correctly numbered contiguous # chunk)
  # if all the springs of group-length is # or ? (then it's a viable chunk/group)
  if all(spring == '#' or spring == '?' for spring in springs[:group]) and \
  (len(springs) == group or springs[group] == '.' or springs[group] == '?'): # AND the springs is just group-length (meaning, at the end of group), OR if the spring after that is '.' or '?'
    return True # return True
  return False

@functools.cache # to memoize this function specifically (this is a decoration)
def countArrangements(springs, groups):
  # checking if it's at all possible by naively summing through the inferred broken springs in the groups
  totalPossibleBroken = sum(groups)
  minPossibleBroken = sum(1 for spring in springs if spring == '#') # oh if you want to use else 0 you have to put the if statement before the for statement
  maxPossibleBroken = sum(1 for spring in springs if spring == '#' or spring == '?')
  if minPossibleBroken > totalPossibleBroken or maxPossibleBroken < totalPossibleBroken:
    return 0

  # if the totalPossibleBroken is zero, and it passes the above check, it necessarily means that
  # the springs variable is either all '.', or is empty; and doesn't need to be checked anymore
  # so it means that this is a viable arrangement
  if totalPossibleBroken == 0:
    return 1

  # now check if the first spring in springs is '.'
  if springs[0] == '.':
    return countArrangements(springs[1:], groups) # if so, just skip it

  # chack if the first spring in springs is '#'
  if springs[0] == '#':
    group = groups[0] # length of the first group
    if checkFirstGroup(springs, group):
      if len(springs) == group: # if the length of springs the same, then it's the end
        return 1 # and it's a viable arrangement
      return countArrangements(springs[group + 1:], groups[1:]) # if not the end, then again, countArrangements again, start ahead
    return 0 # if it doesn't pass checkFirstGroup, it's unviable

  # now if the first spring in springs is '?'
  # check the first case, if it's '.'
  return countArrangements(springs[1:], groups) +\
  countArrangements(('#',) + springs[1:], groups) # plus the second case, if it's '#'
  # it's tuple so you have to use the ('#', ) syntax

def part1(springsRows):
  return sum(countArrangements(*row) for row in springsRows) # *row unpacks into springs and groups

part1(springsRows)
# 21, right for testcase
# 7025, right !!


7025

In [60]:
# day 12 - part 2
# part 2 input adjustments
def part2Adjustments(springsRows, multiplier):
  temp = []
  for i, row in enumerate(springsRows):
    springs, group = row[0], row[1]
    springs += (multiplier - 1) * (('?',) + springs)
    group += (multiplier - 1) * group
    temp.append([springs, group])
  return temp

inputFilename = 'advent-of-code-day12-input'
# inputFilename = 'advent-of-code-day12-testcase'
multiplier = 5
springsRows = part2Adjustments(cleanInput(inputFilename), multiplier)
# springsRows

# we can reuse the logic from part1()
part1(springsRows)
# 525152 right for test case
# 11461095383315, right!! (takes 3" of runtime here)

11461095383315

# scratchpad

In [ ]:
# chatgpt's solution feels to wonky, and doesn't work
# think from first principles man
# btw chatgpt's been worse man

def countArrangements(row, groups, index = 0, currentGroupSize = 0, memo = None):
  if memo is None:
    memo = {} # if there's no memo, then instantiate it

  if index == len(row): # if it /has/ overflowed
    # return 1 if it's a viable arrangement, ie. the groups array is empty and the current group size is zero
    # note that if there's no ? char, then there's no arrangements to be considered, so it'll return zero
    # ie groups is empty BUT currentGroupSize is not
    if not groups and currentGroupSize == 0:
      return 1
    elif len(groups) == 1 and currentGroupSize == groups[0]:
      return 1
    else:
      return 0

  if (index, tuple(groups), currentGroupSize) in memo:
    return memo[(index, tuple(groups), currentGroupSize)]

  if row[index] == '#': # if the current spring is broken,
    nextGroupSize = currentGroupSize + 1 # then the nextGroupSize (for recursive function down the line) is added by one
  elif row[index] == '.': # if it's not broken though,
    nextGroupSize = 0 # then the nextGroupSize is reset
    # (confusion) hmm this seems to assume that there'll be no rows with all '.', because if so it'll count as an arrangement??
  else:
    # assume ? is broken (#)
    if groups and currentGroupSize + 1 <= groups[0]: # if there's still groups, and it doesn't exceed the possible size,
      countBroken = countArrangements(row, groups if currentGroupSize+1 < groups[0] else groups[1:], index + 1, currentGroupSize + 1, memo)
    else:
      countBroken = 0 # else, countBroken is zero
    # assume it's operational (.)
    countOperational = countArrangements(row, groups, index + 1, 0, memo) # currentGroupSize becomes zero.. because it's stopped?
    # update the memo
    memo[(index, tuple(groups), currentGroupSize)] = countBroken + countOperational
    return memo[(index, tuple(groups), currentGroupSize)]

  return countArrangements(row, groups[1:] if groups and nextGroupSize and nextGroupSize == groups[0] else groups, index + 1, nextGroupSize, memo)

def totalArrangements(springsRows):
  total = 0
  for springRow in springsRows:
    row = springRow[0]
    groups = springRow[1]
    # print(row, groups)
    total += countArrangements(row, groups)
  return total

totalArrangements(springRows)


0

In [ ]:
# chatgpt's implementation ()
# i'm gonna try to copy this with /minimal/ looking
def count_arrangements(row, groups, index=0, current_group=0, memo=None):
    if memo is None:
        memo = {}

    # Base case: reached the end of the row
    if index == len(row):
        # Check if all groups are matched and no current group is being formed
        return 1 if not groups and current_group == 0 else 0

    # Use memoization to avoid redundant calculations
    if (index, tuple(groups), current_group) in memo:
        return memo[(index, tuple(groups), current_group)]

    if row[index] == '#':
        # Continue the current group of broken springs
        next_group = current_group + 1
    elif row[index] == '.':
        # Reset the current group
        next_group = 0
    else:  # row[index] is '?'
        # Explore both possibilities: broken or operational spring
        # Case 1: Consider it as broken spring
        if groups and current_group + 1 <= groups[0]:
            count_broken = count_arrangements(row, groups if current_group + 1 < groups[0] else groups[1:], index + 1, current_group + 1, memo)
        else:
            count_broken = 0

        # Case 2: Consider it as operational spring
        count_operational = count_arrangements(row, groups, index + 1, 0, memo)

        # Store the result in memo and return the sum
        memo[(index, tuple(groups), current_group)] = count_broken + count_operational
        return memo[(index, tuple(groups), current_group)]

    # Recursive call for next index
    return count_arrangements(row, groups[1:] if next_group and next_group == groups[0] else groups, index + 1, next_group, memo)

def total_arrangements(data):
    total = 0
    for entry in data:
        row, group_str = entry.split(' ')
        groups = [int(x) for x in group_str.split(',')]
        total += count_arrangements(row, groups)
    return total

# Example usage
data = [
    ".??..??...?##. 1,1,3",
    # ... other rows ...
]
print(total_arrangements(data))


In [ ]:
# a = [1]
# if not a[1:]:
#   print('wiw')


wiw


In [52]:
# (2,) + (1,2)
# tuple(springsRows[0][0])
(1,2) + 4*(1,2)

(1, 2, 1, 2, 1, 2, 1, 2, 1, 2)